In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
import psycopg2
warnings.filterwarnings('ignore')

In [2]:
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

In [3]:
conn=psycopg2.connect(
    dbname="frauddb",
    user="myuser",
    password="mypassword",
    host="postgres",
    port="5432"
)

In [4]:
df=pd.read_sql("select * from transactions_raw",conn)

In [6]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib
import pandas as pd
df = df.drop(['nameorig', 'namedest'], axis=1)
X = df.drop(['isfraud', 'isflaggedfraud'], axis=1)
y = df['isfraud']
categorical = ["type"]
numerical = [col for col in X.columns if col != "type"]

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical),
        ("num", StandardScaler(), numerical)
    ]
)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3, 
    random_state=42, 
    stratify=y
)

In [11]:
from xgboost import XGBClassifier

model = XGBClassifier(
    n_estimators=300,
    max_depth=5,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="logloss"
)

pipeline = Pipeline([
    ("preprocess", preprocess),
    ("model", model)
])

In [12]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['type']),
                                                 ('num', StandardScaler(),
                                                  ['step', 'amount',
                                                   'oldbalanceorg',
                                                   'newbalanceorig',
                                                   'oldbalancedest',
                                                   'newbalancedest'])])),
                ('model',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsamp...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=5, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=300, n_jobs=None,
                               num_parallel_tree=None, random_state=None, ...))])

In [13]:
import os
os.makedirs("model", exist_ok=True)

joblib.dump(pipeline, "model/fraud_pipeline.pkl")
print("Modèle enregistré dans model/fraud_pipeline.pkl")

PermissionError: [Errno 13] Permission denied: 'model/fraud_pipeline.pkl'